In [17]:
import pandas as pd
import numpy as np


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:

offensive = pd.read_excel("/content/drive/MyDrive/pittsburgh-annotation-son-20240613/hate_speech_isik_llm_model_positive_preds_evaluation_offensive_hate_annotation_busra_kadir_merged_20240603_2nd.xlsx")


In [20]:
offensive.rename(columns = {'Hate \nSpeech':"hate-speech"},inplace=True)

In [21]:
import pandas as pd

def resolve_hate_speech(group):
    busra_value = group[group['Annotator'] == 'busra']['hate-speech'].values
    kadir_value = group[group['Annotator'] == 'kadir']['hate-speech'].values
    final_value = group[group['Annotator'] == 'final']['hate-speech'].values

    if len(busra_value) > 0 and len(kadir_value) > 0:
        busra_value = busra_value[0]
        kadir_value = kadir_value[0]
        if busra_value == kadir_value:
            return busra_value
        elif len(final_value) > 0:
            return final_value[0]
        else:
            return None
    else:
        return None

grouped = offensive.groupby('text')
resolved_values = grouped.apply(resolve_hate_speech)
offensive['Resolved_Hate_Speech'] = offensive['text'].map(resolved_values).astype('Int64')
offensive_final = offensive.drop_duplicates(subset='text', keep='first')
offensive_final = offensive_final.reset_index(drop=True)
offensive_final["Resolved_Hate_Speech"].astype(str)
print(offensive_final)

     Annotator      #                                               text  \
0        busra  314.0  @meraldanis @Alone_36_ Sen o kadar vatansevers...   
1        busra  316.0  @meraldanis Erzurum'da kaza geçirmiş. Ne yazık...   
2        busra  321.0  @gulderenvarli Emekçi kürt halkın verdiği verg...   
3        busra  324.0  @meraldanis Bunlari bebek katili savunan diyor...   
4        busra  328.0  @meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...   
...        ...    ...                                                ...   
1076       NaN    NaN                                              agree   
1077       NaN    NaN                                           disagree   
1078       NaN    NaN                                          agree/dis   
1079       NaN    NaN                                              total   
1080       NaN    NaN                                           subtotal   

      Dis\nNeutral  Offense \nNeutral  dis_O_ Relevant  Offense \nRelevant  \
0        

In [22]:
offensive_final["Resolved_Hate_Speech"].value_counts()

Resolved_Hate_Speech
1    612
0    463
Name: count, dtype: Int64

In [23]:
import pandas as pd

# Define the function to resolve values for each annotator
def resolve_annotation(group, column_name):
    busra_value = group[group['Annotator'] == 'busra'][column_name].values
    kadir_value = group[group['Annotator'] == 'kadir'][column_name].values
    final_value = group[group['Annotator'] == 'final'][column_name].values

    if len(busra_value) > 0 and len(kadir_value) > 0:
        busra_value = busra_value[0]
        kadir_value = kadir_value[0]
        if busra_value == kadir_value:
            return busra_value
        elif len(final_value) > 0:
            return final_value[0]
        else:
            return None
    else:
        return None

# Read the data
offensive = pd.read_excel('/content/drive/MyDrive/pittsburgh-annotation-son-20240613/hate_speech_isik_llm_model_positive_preds_evaluation_offensive_hate_annotation_busra_kadir_merged_20240603_2nd.xlsx')

# List of columns to resolve
columns_to_resolve = ['Hate \nSpeech', 'Hate \nEthnicity', 'Hate  \nReligion', 'Hate \nGender']

# Resolve each column and add a new column for the resolved value
for column in columns_to_resolve:
    grouped = offensive.groupby('text')
    resolved_values = grouped.apply(lambda group: resolve_annotation(group, column))
    new_column_name = 'Resolved_' + column.replace(" \n", "_").replace(" ", "_")
    offensive[new_column_name] = offensive['text'].map(resolved_values).astype('Int64')

# Filter the DataFrame to only include rows where 'Resolved_Hate_Speech' is 1
offensive_final = offensive[offensive['Resolved_Hate_Speech'] == 1]

# Drop duplicates and reset index
offensive_final = offensive_final.drop_duplicates(subset='text', keep='first')
offensive_final = offensive_final.reset_index(drop=True)

# Select relevant columns
resolved_columns = ['Resolved_' + column.replace(" \n", "_").replace(" ", "_") for column in columns_to_resolve]
offensive_final = offensive_final[['text'] + resolved_columns]

# Print the final DataFrame
print(offensive_final)

# Save to an Excel file if needed
offensive_final.to_excel('resolved_hate_speech.xlsx', index=False)


                                                  text  Resolved_Hate_Speech  \
0    @meraldanis @Alone_36_ Sen o kadar vatansevers...                     1   
1    @meraldanis Erzurum'da kaza geçirmiş. Ne yazık...                     1   
2    @gulderenvarli Emekçi kürt halkın verdiği verg...                     1   
3    @meraldanis Bunlari bebek katili savunan diyor...                     1   
4    @meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...                     1   
..                                                 ...                   ...   
607  @sakiksirri Dantelli hevallerle mi tehdit ediy...                     1   
608  @sakiksirri Gerçekten de Kürt sorunu yok, geli...                     1   
609  @SamyeliSuhan @sakiksirri Ne kürtçüsü? Bunlar ...                     1   
610     @sakiksirri Aleviden de ülü'l-emr olmaz ki. :)                     1   
611  @KeremFirtina Seni kurtlar vadisinde oynatan O...                     1   

     Resolved_Hate_Ethnicity  Resolved_

In [25]:
offensive_final["Resolved_Hate__Religion"].value_counts()

Resolved_Hate__Religion
0    584
1     27
Name: count, dtype: Int64

In [26]:
# Print the value counts for each of the resolved columns
for column in resolved_columns:
    print(f"Value counts for {column}:")
    print(offensive_final[column].value_counts())
    print("\n")


Value counts for Resolved_Hate_Speech:
Resolved_Hate_Speech
1    612
Name: count, dtype: Int64


Value counts for Resolved_Hate_Ethnicity:
Resolved_Hate_Ethnicity
1    517
0     91
Name: count, dtype: Int64


Value counts for Resolved_Hate__Religion:
Resolved_Hate__Religion
0    584
1     27
Name: count, dtype: Int64


Value counts for Resolved_Hate_Gender:
Resolved_Hate_Gender
0    550
1     62
Name: count, dtype: Int64




In [29]:
offensive_final.dtypes

text                       object
Resolved_Hate_Speech        Int64
Resolved_Hate_Ethnicity     Int64
Resolved_Hate__Religion     Int64
Resolved_Hate_Gender        Int64
dtype: object

In [35]:
import pandas as pd

# Assuming your DataFrame is named 'offensive_final'
# Load your DataFrame (replace this with your actual DataFrame loading code)
# offensive_final = pd.read_csv('your_file.csv')

# Convert all columns to str
offensive_final = offensive_final.astype(str)

# Verify the change
print(offensive_final.dtypes)


text         object
hate         object
ethnicity    object
religion     object
gender       object
dtype: object


In [33]:
import pandas as pd

# Assuming your DataFrame is named 'offensive_final'
# Load your DataFrame (replace this with your actual DataFrame loading code)
# offensive_final = pd.read_csv('your_file.csv')

# Convert all columns to str if necessary
# offensive_final = offensive_final.astype(str)

# Rename the columns
offensive_final.rename(columns={
    'Resolved_Hate_Speech': 'hate',
    'Resolved_Hate_Ethnicity': 'ethnicity',
    'Resolved_Hate__Religion': 'religion',
    'Resolved_Hate_Gender': 'gender'
}, inplace=True)

# Verify the change
print(offensive_final.dtypes)
print(offensive_final.head())


text         object
hate         object
ethnicity    object
religion     object
gender       object
dtype: object
                                                text hate ethnicity religion  \
0  @meraldanis @Alone_36_ Sen o kadar vatansevers...    1         1        0   
1  @meraldanis Erzurum'da kaza geçirmiş. Ne yazık...    1         1        0   
2  @gulderenvarli Emekçi kürt halkın verdiği verg...    1         0        1   
3  @meraldanis Bunlari bebek katili savunan diyor...    1         1        0   
4  @meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...    1         1        0   

  gender  
0      1  
1      0  
2      0  
3      0  
4      0  


In [44]:
offensive_final

,text,hate,ethnicity,religion,gender
0,@meraldanis @Alone_36_ Sen o kadar vatansevers...,1,1,0,1
1,@meraldanis Erzurum'da kaza geçirmiş. Ne yazık...,1,1,0,0
2,@gulderenvarli Emekçi kürt halkın verdiği verg...,1,0,1,0
3,@meraldanis Bunlari bebek katili savunan diyor...,1,1,0,0
4,@meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...,1,1,0,0
...,...,...,...,...,...
607,@sakiksirri Dantelli hevallerle mi tehdit ediy...,1,0,0,1
608,"@sakiksirri Gerçekten de Kürt sorunu yok, geli...",1,1,0,0
609,@SamyeliSuhan @sakiksirri Ne kürtçüsü? Bunlar ...,1,1,0,0
610,@sakiksirri Aleviden de ülü'l-emr olmaz ki. :),1,0,0,0


In [47]:
import pandas as pd

# Assuming 'offensive_final' is already loaded
# Replace empty strings with actual pd.NA
offensive_final = offensive_final.replace('', pd.NA)

# List of columns to convert to integer type
columns_to_convert = offensive_final.columns.difference(['text'])

# Convert columns to nullable integer type
for column in columns_to_convert:
    offensive_final[column] = pd.to_numeric(offensive_final[column], errors='coerce').astype('Int64')

# Drop rows where any column (except 'text') is pd.NA
offensive_final = offensive_final.dropna(subset=columns_to_convert)

# Verify the cleanup
print(offensive_final.isna().sum())

# Show the cleaned DataFrame
print(offensive_final)


text         0
hate         0
ethnicity    0
religion     0
gender       0
dtype: int64
                                                  text  hate  ethnicity  \
0    @meraldanis @Alone_36_ Sen o kadar vatansevers...     1          1   
1    @meraldanis Erzurum'da kaza geçirmiş. Ne yazık...     1          1   
2    @gulderenvarli Emekçi kürt halkın verdiği verg...     1          0   
3    @meraldanis Bunlari bebek katili savunan diyor...     1          1   
4    @meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...     1          1   
..                                                 ...   ...        ...   
607  @sakiksirri Dantelli hevallerle mi tehdit ediy...     1          0   
608  @sakiksirri Gerçekten de Kürt sorunu yok, geli...     1          1   
609  @SamyeliSuhan @sakiksirri Ne kürtçüsü? Bunlar ...     1          1   
610     @sakiksirri Aleviden de ülü'l-emr olmaz ki. :)     1          0   
611  @KeremFirtina Seni kurtlar vadisinde oynatan O...     1          1   

     religi

In [50]:
offensive_final = offensive_final.astype(str)

In [51]:

from sklearn.model_selection import train_test_split

In [52]:
train_df, test_df = train_test_split(offensive_final, test_size=0.2, random_state=42)

In [53]:
train_df.to_json("/content/drive/MyDrive/pittsburgh-annotation/rge_train.json",orient="records",lines=True)

In [54]:
offensive_final.to_json("/content/drive/MyDrive/pittsburgh-annotation/rge.json",orient="records",lines=True)

In [55]:
test_df.to_json("/content/drive/MyDrive/pittsburgh-annotation/rge_test.json",orient="records",lines=True)

In [84]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
import json
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm

class TransformersData(Dataset):
    def __init__(self, examples, tokenizer, max_seq_length=512, with_label=True):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.with_label = with_label

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        encoded_input = self.tokenizer(ex[0], padding="max_length", truncation=True, max_length=self.max_seq_length)
        input_ids = torch.tensor(encoded_input["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(encoded_input["attention_mask"], dtype=torch.long)

        if self.with_label:
            labels = torch.tensor(ex[1], dtype=torch.float)
            return input_ids, attention_mask, labels
        return input_ids, attention_mask

def get_examples(filename, with_label=True):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    examples = []
    for line in lines:
        data = json.loads(line)
        text = str(data["text"])
        if with_label:
            labels = [int(data["ethnicity"]), int(data["religion"]), int(data["gender"])]
            examples.append([text, labels])
        else:
            examples.append([text])
    return examples

class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, dropout=0.5):
        super(MLP, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)

def build_model(train_examples, dev_examples, pretrained_model, tokenizer, n_epochs=10, curr_model_path="temp.pt"):
    encoder = AutoModel.from_pretrained(pretrained_model)
    classifier = MLP(encoder.config.hidden_size, len(label_list))

    train_dataset = TransformersData(train_examples, tokenizer, max_seq_length=max_seq_length)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    dev_dataset = TransformersData(dev_examples, tokenizer, max_seq_length=max_seq_length)
    dev_dataloader = DataLoader(dataset=dev_dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    encoder.to(device)
    classifier.to(device)

    optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(classifier.parameters()), lr=learning_rate)
    criterion = torch.nn.BCEWithLogitsLoss()

    best_score = -1e6
    for epoch in range(n_epochs):
        encoder.train()
        classifier.train()
        total_loss = 0

        for batch in tqdm(train_dataloader, desc="Training"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            optimizer.zero_grad()
            embeddings = encoder(input_ids=input_ids, attention_mask=attention_mask)[1]
            outputs = classifier(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dataloader)

        encoder.eval()
        classifier.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in dev_dataloader:
                input_ids, attention_mask, labels = [b.to(device) for b in batch]
                embeddings = encoder(input_ids=input_ids, attention_mask=attention_mask)[1]
                outputs = classifier(embeddings)
                preds = torch.sigmoid(outputs).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(labels.cpu().numpy())

        all_preds = np.vstack(all_preds)
        all_labels = np.vstack(all_labels)

        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, (all_preds > 0.5).astype(int), average='macro')
        accuracy = accuracy_score(all_labels, (all_preds > 0.5).astype(int))

        print(f"Epoch {epoch + 1} - Loss: {avg_train_loss:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - F1: {f1:.4f} - Accuracy: {accuracy:.4f}")

        if f1 > best_score:
            best_score = f1
            print("Saving model!")
            torch.save(classifier.state_dict(), repo_path + "/models/classifier_" + curr_model_path)
            encoder_to_save = encoder.module if hasattr(encoder, 'module') else encoder
            torch.save(encoder_to_save.state_dict(), repo_path + "/models/encoder_" + curr_model_path)
            tokenizer.save_pretrained(repo_path + "/models/tokenizer_" + curr_model_path)

    return encoder, classifier

if __name__ == "__main__":
    pretrained_model = "dbmdz/bert-base-turkish-128k-cased"
    seed = 42
    max_seq_length = 512
    batch_size = 8
    dev_ratio = 0.1
    learning_rate = 2e-5
    n_epochs = 5
    model_path = "model"

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    train_filename = "/content/drive/MyDrive/pittsburgh-annotation/rge_train.json"
    test_filename = "/content/drive/MyDrive/pittsburgh-annotation/rge_test.json"
    label_list = ["ethnicity", "religion", "gender"]

    train_examples = get_examples(train_filename)
    random.shuffle(train_examples)
    dev_split = int(len(train_examples) * dev_ratio)
    dev_examples = train_examples[:dev_split]
    train_examples = train_examples[dev_split:]

    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

    encoder, classifier = build_model(train_examples, dev_examples, pretrained_model, tokenizer, n_epochs, model_path)


Training: 100%|██████████| 55/55 [00:09<00:00,  5.84it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1 - Loss: 0.3742 - Precision: 0.2917 - Recall: 0.3333 - F1: 0.3111 - Accuracy: 0.8542
Saving model!


Training: 100%|██████████| 55/55 [00:09<00:00,  5.83it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2 - Loss: 0.3027 - Precision: 0.2917 - Recall: 0.3333 - F1: 0.3111 - Accuracy: 0.8542


Training: 100%|██████████| 55/55 [00:09<00:00,  5.84it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Training: 100%|██████████| 55/55 [00:09<00:00,  5.84it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 4 - Loss: 0.2107 - Precision: 0.6512 - Recall: 0.6587 - F1: 0.6549 - Accuracy: 0.8958
Saving model!


Training: 100%|██████████| 55/55 [00:09<00:00,  5.83it/s]


Epoch 5 - Loss: 0.1367 - Precision: 0.6512 - Recall: 0.6587 - F1: 0.6549 - Accuracy: 0.8958


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
import os
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
import json
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm

class TransformersData(Dataset):
    def __init__(self, examples, tokenizer, max_seq_length=512, with_label=True):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.with_label = with_label

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        encoded_input = self.tokenizer(ex[0], padding="max_length", truncation=True, max_length=self.max_seq_length)
        input_ids = torch.tensor(encoded_input["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(encoded_input["attention_mask"], dtype=torch.long)

        if self.with_label:
            labels = torch.tensor(ex[1], dtype=torch.float)
            return input_ids, attention_mask, labels
        return input_ids, attention_mask

def get_examples(filename, with_label=True):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    examples = []
    for line in lines:
        data = json.loads(line)
        text = str(data["text"])
        if with_label:
            labels = [int(data["ethnicity"]), int(data["religion"]), int(data["gender"])]
            examples.append([text, labels])
        else:
            examples.append([text])
    return examples

class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, dropout=0.5):
        super(MLP, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)

def build_model(train_examples, dev_examples, pretrained_model, tokenizer, n_epochs=10, curr_model_path="temp.pt"):
    encoder = AutoModel.from_pretrained(pretrained_model)
    classifier = MLP(encoder.config.hidden_size, len(label_list))

    train_dataset = TransformersData(train_examples, tokenizer, max_seq_length=max_seq_length)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    dev_dataset = TransformersData(dev_examples, tokenizer, max_seq_length=max_seq_length)
    dev_dataloader = DataLoader(dataset=dev_dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    encoder.to(device)
    classifier.to(device)

    optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(classifier.parameters()), lr=learning_rate)
    criterion = torch.nn.BCEWithLogitsLoss()

    best_score = -1e6
    for epoch in range(n_epochs):
        encoder.train()
        classifier.train()
        total_loss = 0

        for batch in tqdm(train_dataloader, desc="Training"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            optimizer.zero_grad()
            embeddings = encoder(input_ids=input_ids, attention_mask=attention_mask)[1]
            outputs = classifier(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dataloader)

        encoder.eval()
        classifier.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in dev_dataloader:
                input_ids, attention_mask, labels = [b.to(device) for b in batch]
                embeddings = encoder(input_ids=input_ids, attention_mask=attention_mask)[1]
                outputs = classifier(embeddings)
                preds = torch.sigmoid(outputs).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(labels.cpu().numpy())

        all_preds = np.vstack(all_preds)
        all_labels = np.vstack(all_labels)

        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, (all_preds > 0.5).astype(int), average='macro', zero_division=0
        )
        accuracy = accuracy_score(all_labels, (all_preds > 0.5).astype(int))

        print(f"Epoch {epoch + 1} - Loss: {avg_train_loss:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - F1: {f1:.4f} - Accuracy: {accuracy:.4f}")

        if f1 > best_score:
            best_score = f1
            print("Saving model!")
            model_dir = "content/drive/MyDrive/pittsburgh-annotation/models"
            os.makedirs(model_dir, exist_ok=True)
            classifier_name = os.path.join(model_dir, f"classifier_hate_rge_dbmdz_bert-base-turkish-128k-cased_{max_seq_length}_{batch_size}_{learning_rate:.1e}_{seed}.pt")
            encoder_name = os.path.join(model_dir, f"encoder_hate_rge_dbmdz_bert-base-turkish-128k-cased_{max_seq_length}_{batch_size}_{learning_rate:.1e}_{seed}.pt")
            torch.save(classifier.state_dict(), classifier_name)
            encoder_to_save = encoder.module if hasattr(encoder, 'module') else encoder
            torch.save(encoder_to_save.state_dict(), encoder_name)
            tokenizer.save_pretrained(os.path.join(model_dir, f"tokenizer_hate_rge_dbmdz_bert-base-turkish-128k-cased_{max_seq_length}_{batch_size}_{learning_rate:.1e}_{seed}"))

    return encoder, classifier

if __name__ == "__main__":
    pretrained_model = "dbmdz/bert-base-turkish-128k-cased"
    seed = 42
    max_seq_length = 512
    batch_size = 8
    dev_ratio = 0.1
    learning_rate = 2e-5
    n_epochs = 5

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    train_filename = "/content/drive/MyDrive/pittsburgh-annotation/rge_train.json"
    test_filename = "/content/drive/MyDrive/pittsburgh-annotation/rge_test.json"
    label_list = ["ethnicity", "religion", "gender"]

    train_examples = get_examples(train_filename)
    random.shuffle(train_examples)
    dev_split = int(len(train_examples) * dev_ratio)
    dev_examples = train_examples[:dev_split]
    train_examples = train_examples[dev_split:]

    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

    encoder, classifier = build_model(train_examples, dev_examples, pretrained_model, tokenizer, n_epochs)


Training: 100%|██████████| 55/55 [00:09<00:00,  5.83it/s]


Epoch 1 - Loss: 0.3742 - Precision: 0.2917 - Recall: 0.3333 - F1: 0.3111 - Accuracy: 0.8542
Saving model!


Training: 100%|██████████| 55/55 [00:09<00:00,  5.87it/s]


Epoch 2 - Loss: 0.3027 - Precision: 0.2917 - Recall: 0.3333 - F1: 0.3111 - Accuracy: 0.8542


Training: 100%|██████████| 55/55 [00:09<00:00,  5.87it/s]


Epoch 3 - Loss: 0.2801 - Precision: 0.2979 - Recall: 0.3333 - F1: 0.3146 - Accuracy: 0.8542
Saving model!


Training: 100%|██████████| 55/55 [00:09<00:00,  5.84it/s]


Epoch 4 - Loss: 0.2107 - Precision: 0.6512 - Recall: 0.6587 - F1: 0.6549 - Accuracy: 0.8958
Saving model!


Training: 100%|██████████| 55/55 [00:09<00:00,  5.84it/s]


Epoch 5 - Loss: 0.1367 - Precision: 0.6512 - Recall: 0.6587 - F1: 0.6549 - Accuracy: 0.8958
